## 4.9 Parser Generators

### 4.9.1

> Write a `Yacc` program that takes boolean expressions as input and produces the truth value of the expressions.

Compile:
```
yacc -d 4.9.1.y && lex 4.9.1.l && cc -o syn.out y.tab.c lex.yy.c -ll
```

4.9.1.l:
```
%{
/*
  AND, OR, NOT, LB, RB, TRUE, FALSE, NEWLINE
*/
#include "y.tab.h"
int yylval;
%}

delim [ \t]
ws {delim}+
newline \n
lb \(
rb \)

%%

{ws}      { }
{newline} { return (NEWLINE); }
and       { return (AND); }
or        { return (OR); }
not       { return (NOT); }
{lb}      { return (LB); }
{rb}      { return (RB); }
true      { yylval = 1; return (TRUE); }
false     { yylval = 0; return (FALSE); }

%%

int yywrap() {
    return 1;
}

void yyerror(char const *s) {
    fprintf(stderr, "%s\n", s);
}
```

4.9.1.y:
```
%{
#include <ctype.h>
#include <stdio.h>
%}

%token AND OR NOT LB RB TRUE FALSE NEWLINE

%%

lines : lines bexpr NEWLINE { if ($2) puts("true"); else puts("false"); }
      | lines NEWLINE
      |
      ;
bexpr : bexpr OR bterm { $ = $1 || $3; }
      | bterm          { $ = $1; }
      ;
bterm : bterm AND bfactor { $ = $1 && $3; }
      | bfactor           { $ = $1; }
      ;
bfactor : NOT bfactor { $ = !$2; }
        | LB bexpr RB { $ = $2; }
        | TRUE
        | FALSE
        ;

%%

int main() {
    yyparse();
    return 0;
}
```

### 4.9.2

> Write a `Yacc` program that takes lists and produces as output a linear representation of the same list; i.e., a single list of the elements, in the same order that they appear in the input.

```
%{
/*
  COMMA, LB, RB, CHARA
*/
#include "y.tab.h"
char yylval;
%}

delim [ \t\n]
ws {delim}+
newline \n
lb \(
rb \)
chara [a-zA-Z]

%%

{ws}      { }
,         { return (COMMA); }
{lb}      { return (LB); }
{rb}      { return (RB); }
{chara}   { yylval = yytext[0]; return (CHARA); }

%%

int yywrap() {
    return 1;
}

void yyerror(char const *s) {
    fprintf(stderr, "%s\n", s);
}
```

```
%{
#include <ctype.h>
#include <stdio.h>
#include <stdlib.h>

struct List {
    char val;
    struct List* next;
};

struct List* head;
struct List* tail;

struct List* createNode(char val) {
    struct List* node = (struct List*) malloc(sizeof(struct List));
    node->val = val;
    node->next = NULL;
    return node;
}

struct List* destroyNode(struct List* node) {
    struct List* next;
    while (node) {
        next = node->next;
        free(node);
        node = next;
    }
}
%}

%token COMMA LB RB CHARA

%%

S : LB L RB
  | CHARA {
            if (tail == NULL) {
                head = createNode(yylval);
                tail = head;
            } else {
                tail->next = createNode(yyval);
                tail = tail->next;
            }
        }
  ;
L : L COMMA S
  | S
  ;

%%

int main() {
    int first;
    yyparse();
    first = 1;
    putchar('(');
    tail = head;
    while (tail) {
        if (first) {
            first = 0;
        } else {
            printf(", ");
        }
        putchar(tail->val);
        tail = tail->next;
    }
    puts(")\n");
    destroyNode(head);
    return 0;
}
```

### 4.9.3

> Write a `Yacc` program that tells whether its input is a _palindrome_.

Palindrome is not LALR(1).

### 4.9.4

> Write a `Yacc` program that takes regular expressions and produces as output a transition table for a nondeterministic finite automaton recognizing the same language.